In [ ]:
# Cell purpose: Utility / preparation
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import PIL
from PIL import Image

In [ ]:
# Cell purpose: Configure Detectron2 and build the inference predictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
# Cell purpose: Load/register COCO-format ground-truth annotations and prepare dataset
from detectron2.data.datasets import register_coco_instances
register_coco_instances("DDHxp1_train", {}, "path/to/your/gt_annotations", "path/to/your/images_dir")

In [ ]:
# Cell purpose: Configure Detectron2 and build the inference predictor
from detectron2.data import MetadataCatalog
keypoint_names = ['rtacetab1', 'rtacetab2', 'rtilleum', 'rtfemur1', 'rtfemur2', 'ltacetab1','ltacetab2','ltilleum','ltfemur1','ltfemur2']
keypoint_flip_map = [('rtacetab1', 'ltacetab1'), ('rtilleum', 'ltilleum'), ('rtacetab2', 'ltacetab2'),('rtfemur1','ltfemur1'),('rtfemur2','ltfemur2')]
keypoint_connection_rules = [('rtacetab1', 'rtacetab2',(0,255,255)), ('rtfemur1', 'rtfemur2',(0,100,100)), ('rtilleum', 'ltilleum',(0,255,0)), ('ltacetab1', 'ltacetab2',(255,255,0)), ('ltfemur1', 'ltfemur2',(255,0,255))]
from detectron2.data import MetadataCatalog
classes = MetadataCatalog.get("DDHxp1_train").thing_classes = ["DDHxp1"]
metadata = MetadataCatalog.get("DDHxp1_train")
MetadataCatalog.get("DDHxp1_train").thing_classes = ["DDHxp1"]
MetadataCatalog.get("DDHxp1_train").thing_dataset_id_to_contiguous_id = {1:0}
MetadataCatalog.get("DDHxp1_train").keypoint_names = keypoint_names
MetadataCatalog.get("DDHxp1_train").keypoint_flip_map = keypoint_flip_map
MetadataCatalog.get("DDHxp1_train").keypoint_connection_rules = keypoint_connection_rules
MetadataCatalog.get("DDHXp1_train").evaluator_type="coco"

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode, Keypoints
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
cfg = get_cfg()


cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "path/to/your/path"  # path to the model we just trained
cfg.MODEL.DEVICE = "cuda"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # hand
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 10
cfg.TEST.KEYPOINT_OKS_SIGMAS = np.ones((10, 1), dtype=float).tolist()

predictor = DefaultPredictor(cfg)
im = cv2.imread("path/to/your/path")
outputs = predictor(im)

v = Visualizer(im[:, :, ::-1],
                   metadata, 
                   scale=0.8, 
                       )
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (14, 10))
plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Cell purpose: Utility / preparation
import os, cv2, matplotlib.pyplot as plt
from tqdm import tqdm                     # progress bar (optional)

# imagesfolder
SRC_DIR  = "path/to/your/images_dir"
# visualizationsavefolder
DST_DIR  = "path/to/your/output_overlays"
os.makedirs(DST_DIR, exist_ok=True)

# ---- -------------------------------------------------
for fname in tqdm(sorted(os.listdir(SRC_DIR))):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
        continue

    # 1) imagesload
    path = os.path.join(SRC_DIR, fname)
    im   = cv2.imread(path)

    # 2) inference  -----------------------------------------------------
    outs = predictor(im)                           # works on GPU or CPU
    inst = outs["instances"].to("cpu")             # do visualization on CPU
    inst = inst[inst.pred_boxes.nonempty()]        # filter out zero-area boxes

 # 3) 1 -------------------------------
    if len(inst) > 0:
        best = int(inst.scores.argmax()) 
        inst = inst[[best]]# extract a single row by tensor index
    else:
 # imagessave
        vis_img_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        save_path = os.path.join(DST_DIR, f"{os.path.splitext(fname)[0]}_vis.jpg")
        cv2.imwrite(save_path, cv2.cvtColor(vis_img_rgb, cv2.COLOR_RGB2BGR))
        continue

 # 4) visualization（1 ） --------------------------
    v = Visualizer(im[:, :, ::-1], metadata, scale=0.8)
    v = v.draw_instance_predictions(inst)
    vis_img_rgb = v.get_image()[:, :, ::-1]        # RGB

 # 5) Notebook 
    plt.figure(figsize=(14, 10))
    plt.imshow(vis_img_rgb)
    plt.axis("off")
    plt.show()

 # 6) save（BGR ） -------------------------------
    save_path = os.path.join(
        DST_DIR, f"{os.path.splitext(fname)[0]}_vis.jpg"
    )
    cv2.imwrite(save_path, cv2.cvtColor(vis_img_rgb, cv2.COLOR_RGB2BGR))

In [ ]:
# Cell purpose: Utility / preparation
#extract keypoint coordinates 
Ins = outputs.get('instances')
Insdict = (Ins.__dict__)
fielddict = Insdict.get('_fields')
coordkeyp = fielddict.get('pred_keypoints')
coordkeyp = coordkeyp.cpu()
coordkeyp = coordkeyp.numpy()
coordkeyp2 = np.delete(coordkeyp, 2, 2)
coordkeyp2 = coordkeyp2[0]

In [ ]:
# Cell purpose: Utility / preparation
#keypoint coordinates
rtacetab1coord = coordkeyp2[0]
rtacetab2coord = coordkeyp2[1]
rtilleumcoord = coordkeyp2[2]
rtfemur1coord = coordkeyp2[3]
rtfemur2coord = coordkeyp2[4]
ltacetab1coord = coordkeyp2[5]
ltacetab2coord = coordkeyp2[6]
ltilleumcoord = coordkeyp2[7]
ltfemur1coord = coordkeyp2[8]
ltfemur2coord = coordkeyp2[9]

In [ ]:
# Cell purpose: Render and save acetabular angle auxiliary-line images
import os, cv2, numpy as np, matplotlib.pyplot as plt, pathlib
from tqdm import tqdm

SRC_DIR = "path/to/your/images_dir"
DST_DIR = "path/to/your/output_ihdi_lines"
pathlib.Path(DST_DIR).mkdir(exist_ok=True)

eps = 1e-6  # horizontal check / numerical stability

for fname in tqdm(sorted(os.listdir(SRC_DIR))):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
        continue

    # ---------- 1) imagesload & inference ----------
    path = os.path.join(SRC_DIR, fname)
    im   = cv2.imread(path)
    if im is None:
        print(f"{fname}: failed to read image"); continue

    outs = predictor(im)

 # ---------- 2) 1 ----------
    inst = outs["instances"].to("cpu")
    inst = inst[inst.pred_boxes.nonempty()]   # filter zero-area boxes
    if len(inst) == 0:
        print(f"{fname}: no instance");  continue
    inst = inst[[int(inst.scores.argmax())]]
    kps  = inst.pred_keypoints.numpy()[0, :, :2]  # shape (10,2)

 # ---------- 3) keypoints ----------
    (rtacetab1coord, rtacetab2coord, rtilleumcoord,
     rtfemur1coord,  rtfemur2coord,
     ltacetab1coord, ltacetab2coord, ltilleumcoord,
     ltfemur1coord,  ltfemur2coord) = kps

    rtfemurpoint = (rtfemur1coord + rtfemur2coord) / 2
    ltfemurpoint = (ltfemur1coord + ltfemur2coord) / 2

    # -------------- 4) IHDI auxiliary linescompute ---------------
    p1, p2 = rtilleumcoord, ltilleumcoord  # Two points defining H-line (right and left iliac points)
    if np.linalg.norm(p2 - p1) < eps:
        print(f"{fname}: invalid H-line (p1≈p2)"); continue

 # H-line ax + by + c = 0
    a = p2[1] - p1[1]
    b = p1[0] - p2[0]
    c = p1[1]*p2[0] - p1[0]*p2[1]
    denom = a*a + b*b
    if denom < eps:
        print(f"{fname}: degenerate H-line"); continue

    def foot(pt):
        x = (b*b*pt[0] - a*b*pt[1] - a*c) / denom
        y = (a*a*pt[1] - a*b*pt[0] - b*c) / denom
        return np.array([x, y], dtype=np.float32)

    ltaxispoint = foot(ltacetab1coord)  # perpendicular foot from left acetabular point
    rtaxispoint = foot(rtacetab1coord)  # perpendicular foot from right acetabular point

    if np.any(~np.isfinite(ltaxispoint)) or np.any(~np.isfinite(rtaxispoint)):
        print(f"{fname}: NaN in foot points"); continue

    h, w, _ = im.shape

    # ---------- 5) draw ----------
    fig = plt.figure(dpi=200, figsize=(6, 6))
    ax  = fig.add_subplot(1, 1, 1)

 # （）
    ax.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), alpha=0.9, zorder=0)
    ax.set_xlim(0, w); ax.set_ylim(0, h)
    ax.invert_yaxis(); ax.set_aspect('equal')
    ax.axis("off")

 # lines（）
    p3, p4 = rtaxispoint, ltaxispoint
    a1 = (p2[1] - p1[1]) / (p2[0] - p1[0])  # H-line slope（assumes H-line is not vertical）
    b1 = p1[1] - a1 * p1[0]
    x_all = np.arange(0, w, 1)

    if abs(a1) < eps:
 # --- H-line : P-line ---
        ax.axhline(b1, c='w', zorder=1)

        # P-lines: x = const
        ax.axvline(p3[0], c='w', zorder=1)  # 90° right
        ax.axvline(p4[0], c='w', zorder=1)  # 90° left

        
 # 45° lines（ ±1）— lines
        a4, a5 = -1.0, 1.0
        b4 = p3[1] - a4 * p3[0]
        b5 = p4[1] - a5 * p4[0]

 # ：rtaxispoint [0, p3.x] lines
        x0_r, x1_r = 0.0, float(np.clip(p3[0], 0, w))
        ax.plot([x0_r, x1_r], [a4*x0_r + b4, a4*x1_r + b4], c='w', zorder=1)

 # ：ltaxispoint [p4.x, w] lines
        x0_l, x1_l = float(np.clip(p4[0], 0, w)), float(w)
        ax.plot([x0_l, x1_l], [a5*x0_l + b5, a5*x1_l + b5], c='w', zorder=1)
    else:
 # --- ---
        a2 = -1.0 / a1
        a4 = (a1 - 1.0) / (1.0 + a1)
        a5 = (a1 + 1.0) / (1.0 - a1)
        b2 = p3[1] - a2 * p3[0]
        b3 = p4[1] - a2 * p4[0]
        b4 = p3[1] - a4 * p3[0]
        b5 = p4[1] - a5 * p4[0]

        ax.plot(x_all, a1 * x_all + b1, c='w', zorder=1)                    # H-line
        ax.plot(x_all, a2 * x_all + b2, c='w', zorder=1)                    # 90° right
        ax.plot(x_all, a2 * x_all + b3, c='w', zorder=1)                    # 90° left
        ax.plot(np.arange(0, p3[0], 1), a4*np.arange(0, p3[0], 1)+b4, c='w', zorder=1)  # 45° right
        ax.plot(np.arange(p4[0], w, 1), a5*np.arange(p4[0], w, 1)+b5, c='w', zorder=1)  # 45° left

 # （）
    pts = np.vstack([ltaxispoint, rtaxispoint, rtfemurpoint, ltfemurpoint])
    ax.scatter(pts[:, 0], pts[:, 1], c='k', s=6, zorder=2)

 # ---------- 6) & save ----------
 # plt.show() # 
    save_path = os.path.join(DST_DIR, f"{os.path.splitext(fname)[0]}_ihdi.png")
    fig.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

In [ ]:
# Cell purpose: Measure inference time on GPU/CPU
# ================================================================
# αIHDI CSV export（＆ fp==ax → IHDI=1 ）
# ＋ CPU/GPU inferencetimemeasure
# ================================================================
import os, time, pathlib
import cv2, numpy as np, pandas as pd
import torch
from tqdm import tqdm
# from detectron2.engine import DefaultPredictor # 

SRC_DIR = "path/to/your/images_dir"
DST_CSV = "result_otherhosp1_0907.csv"
pathlib.Path(SRC_DIR).mkdir(exist_ok=True)

eps = 1e-6  # for numerical stability

def calc_alpha_IHDI(coords, eps=1e-6):
    """
    coords: (10,2) = [rt1, rt2, rtil, rtf1, rtf2, lt1, lt2, ltil, ltf1, ltf2]
    return: (right_alpha, left_alpha, right_IHDI, left_IHDI)
            alpha is float or np.nan; IHDI is 1..4 or 'error'
    """
    (rt1, rt2, rtil, rtf1, rtf2,
     lt1, lt2, ltil, ltf1, ltf2) = coords

 # ---- α（）----
    def ang(v1, v2):
        n1 = np.linalg.norm(v1); n2 = np.linalg.norm(v2)
        if n1 < eps or n2 < eps:
            return np.nan
        cosv = np.dot(v1, v2) / (n1 * n2)
        cosv = float(np.clip(cosv, -1.0, 1.0))
        return np.degrees(np.arccos(cosv))

    hdir = ltil - rtil           # H-line direction (horizontal allowed; zero only when iliac points coincide)
    ra = ang(rt2 - rt1, hdir)
    la = ang(lt2 - lt1, -hdir)   # align orientation with one side (legacy)

 # ---- H-line: ax + by + c = 0（/）----
    a = ltil[1] - rtil[1]
    b = rtil[0] - ltil[0]
    c = rtil[1] * ltil[0] - rtil[0] * ltil[1]
    denom = a*a + b*b
    if denom < eps:
 # 
        return ra, la, "error", "error"

    def foot(p):
        x = (b*b*p[0] - a*b*p[1] - a*c) / denom
        y = (a*a*p[1] - a*b*p[0] - b*c) / denom
        return np.array([x, y], dtype=float)

    rax, lax = foot(rt1), foot(lt1)
    rtp = (rtf1 + rtf2) / 2.0
    ltp = (ltf1 + ltf2) / 2.0

 # ---- IHDI （fp==ax → Grade 1 ）----
    thr = -1.0 / np.sqrt(2.0)  # cos 135°
    def ihdi(ax, il, fp, eps=1e-6):
 # H-line ： Grade 4
        side = a*fp[0] + b*fp[1] + c
        if side > 1e-9:
            return 4

 # ★ fp Grade 1
        if np.linalg.norm(fp - ax) < eps:
            return 1

        v1 = ax - il
        v2 = ax - fp
        n1 = np.linalg.norm(v1); n2 = np.linalg.norm(v2)

 # Grade 1（ np.nan ）
        if n1 < eps or n2 < eps or not np.isfinite(n1) or not np.isfinite(n2):
            return 1

        loc = np.dot(v1, v2) / (n1 * n2)   # cos θ
        loc = float(np.clip(loc, -1.0, 1.0))

        if 0 >= loc > thr:     # greater than 90° and less than 135°
            return 2
        elif thr >= loc >= -1: # 135°〜180°
            return 3
        else:                  # 0°〜90°
            return 1

    rI = ihdi(rax, rtil, rtp, eps=eps)
    lI = ihdi(lax, ltil, ltp, eps=eps)
    return ra, la, rI, lI

# ================================================================
# 1) αIHDI CSV export（GPU inference）
# ================================================================
records = []
t_start_gpu = time.time()
num_imgs_gpu = 0

for fname in tqdm(sorted(os.listdir(SRC_DIR))):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
        continue

    path = os.path.join(SRC_DIR, fname)
    im = cv2.imread(path)
    if im is None:
        records.append({"file": fname,
                        "right_alpha": np.nan, "left_alpha": np.nan,
                        "right_IHDI": "error", "left_IHDI": "error"})
        continue

    outs = predictor(im)
    inst = outs["instances"].to("cpu")
    inst = inst[inst.pred_boxes.nonempty()]
    if len(inst) == 0:
        records.append({"file": fname,
                        "right_alpha": np.nan, "left_alpha": np.nan,
                        "right_IHDI": "error", "left_IHDI": "error"})
        continue

    inst = inst[[int(inst.scores.argmax())]]
    kps  = inst.pred_keypoints.numpy()[0, :, :2]
    ra, la, rI, lI = calc_alpha_IHDI(kps, eps=eps)

    records.append({"file": fname,
                    "right_alpha": ra, "left_alpha": la,
                    "right_IHDI": rI, "left_IHDI": lI})
    num_imgs_gpu += 1

t_gpu = time.time() - t_start_gpu

df = pd.DataFrame(records)
df.to_csv(DST_CSV, index=False, encoding="utf-8-sig")
print(f"✓ αIHDI {DST_CSV} save (GPU inference {t_gpu:.2f}s, {num_imgs_gpu} images)")

# ================================================================
# 2) CPU inferencetimemeasure
# ================================================================
torch.set_num_threads(2)  # tune as needed
cfg_cpu = cfg.clone()
cfg_cpu.MODEL.DEVICE = "cpu"
predictor_cpu = DefaultPredictor(cfg_cpu)

t_start_cpu = time.time()
num_imgs_cpu = 0
for fname in sorted(os.listdir(SRC_DIR)):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
        continue
    im = cv2.imread(os.path.join(SRC_DIR, fname))
    if im is None:
        continue
    _ = predictor_cpu(im)
    num_imgs_cpu += 1
t_cpu = time.time() - t_start_cpu

if num_imgs_cpu > 0:
    print(f"path/to/your/path")
    print(f"GPU  total {t_gpu:.2f} s  → {t_gpu/max(num_imgs_gpu,1):.3f} s / image")
    print(f"CPU  total {t_cpu:.2f} s  → {t_cpu/num_imgs_cpu:.3f} s / image")
else:
    print("path/to/your/images_dir")

In [ ]:
# Cell purpose: Configure Detectron2 and build the inference predictor
# ================================================================
# （）
# ================================================================
SRC_DIR   = "path/to/your/images_dir"               # test image folder (expected to contain files referenced by GT 'file_name')
GT_JSON   = "path/to/your/gt_annotations"     # COCO annotations for GT
WEIGHTS   = "path/to/your/path"   # 25k checkpoint (example)
OUT_DIR   = "overlay_gt_pred_otherhosp1_0811"                    # output directory for overlay images
CSV_PATH  = "gt_pred_pairwise_error_otherhosp1_0811.csv"         # output path for error CSV

# export（）：COCO、GTIDCOCOsave True
SAVE_COCO_RESULTS = True
SAVE_COCO_FULL    = True
COCO_RESULTS_PATH = "pred_results_otherhosp1_coco_0811.json"     # COCO results format (list of annotations only)
COCO_FULL_PATH    = "predictions_mirror_otherhosp1_full_0811.json"  # full COCO with the same IDs as GT

# ================================================================
# 
# ================================================================
import os, json, cv2, numpy as np
from pathlib import Path
import pandas as pd

import torch
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2 import model_zoo
from detectron2.data import MetadataCatalog
from detectron2.evaluation.coco_evaluation import instances_to_coco_json

# ================================================================
# （keypoints）
# ================================================================
keypoint_names = [
    'rtacetab1', 'rtacetab2', 'rtilleum', 'rtfemur1', 'rtfemur2',
    'ltacetab1','ltacetab2','ltilleum','ltfemur1','ltfemur2'
]
keypoint_flip_map = [
    ('rtacetab1', 'ltacetab1'),
    ('rtilleum',  'ltilleum'),
    ('rtacetab2', 'ltacetab2'),
    ('rtfemur1',  'ltfemur1'),
    ('rtfemur2',  'ltfemur2'),
]
# visualization（BGR）※、
#keypoint_connection_rules = [
  #  ('rtacetab1', 'rtacetab2', (0,255,255)),
  #  ('rtfemur1',  'rtfemur2',  (0,100,100)),
   # ('ltacetab1', 'ltacetab2', (255,255,0)),
  #  ('ltfemur1',  'ltfemur2',  (255,0,255)),
#]

# （）
META_NAME = "DDHxp1_train"
MetadataCatalog.get(META_NAME).thing_classes = ["DDHxp1"]
MetadataCatalog.get(META_NAME).thing_dataset_id_to_contiguous_id = {1: 0}
MetadataCatalog.get(META_NAME).keypoint_names = keypoint_names
MetadataCatalog.get(META_NAME).keypoint_flip_map = keypoint_flip_map
#MetadataCatalog.get(META_NAME).keypoint_connection_rules = keypoint_connection_rules
MetadataCatalog.get(META_NAME).evaluator_type = "coco"

# ================================================================
# Predictor
# ================================================================
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
cfg.MODEL.WEIGHTS = WEIGHTS
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 10
# OKS（evaluation）。1（：0.05）
cfg.TEST.KEYPOINT_OKS_SIGMAS = [0.05] * 10

predictor = DefaultPredictor(cfg)

# ================================================================
# （OKSIoU）
# ================================================================
def oks(gt_kps, pred_kps, area, sigmas):
    """
    gt_kps, pred_kps: (K,2) coordinates (set invisible GT keypoints to NaN to ignore)
    area: GT bbox area (COCO s)
    sigmas: (K,) per-keypoint sigma values
    """
    vis = ~np.isnan(gt_kps[:, 0])
    if vis.sum() == 0:
        return 0.0
    d2 = ((gt_kps[vis] - pred_kps[vis]) ** 2).sum(axis=1)
    vars = (sigmas[vis] * 2) ** 2
    oks_i = np.exp(-d2 / (2 * area * vars + 1e-12))
    return float(oks_i.mean())

def iou_xyxy(a, b):
    xa1, ya1, xa2, ya2 = a
    xb1, yb1, xb2, yb2 = b
    inter = max(0, min(xa2, xb2) - max(xa1, xb1)) * max(0, min(ya2, yb2) - max(ya1, yb1))
    area_a = max(0, xa2 - xa1) * max(0, ya2 - ya1)
    area_b = max(0, xb2 - xb1) * max(0, yb2 - yb1)
    union = area_a + area_b - inter + 1e-12
    return inter / union

# ================================================================
# ：keypointsdraw
# - GT : （）
# - Pred: （）
# ================================================================
def _ensure_bgr(img):
    if img.ndim == 2 or (img.ndim == 3 and img.shape[2] == 1):
        return cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    return img

def draw_points_bw(img, kps, style="filled_circle", radius=6, thickness=2, halo=True):
    """
    Markers for black-and-white printing
      style:
        - "filled_circle": white filled circle (for GT)
        - "cross": white cross (for predictions)
      radius: marker size
      thickness: line thickness (outline or cross)
      halo: if True, add black outline (halo) to improve visibility
    """
    img = _ensure_bgr(img)
    for x, y in kps:
        if np.isnan(x) or np.isnan(y):
            continue
        c = (int(round(x)), int(round(y)))

        if style == "filled_circle":
            if halo:
                cv2.circle(img, c, radius+1, (0, 0, 0), 2, lineType=cv2.LINE_AA)  # black outline
            cv2.circle(img, c, radius, (255, 255, 255), -1, lineType=cv2.LINE_AA)  # white filled

        elif style == "cross":
 # 
            x0, y0 = c
            pL = (x0 - radius, y0)
            pR = (x0 + radius, y0)
            pT = (x0, y0 - radius)
            pB = (x0, y0 + radius)
            if halo:
 # （）
                cv2.line(img, pL, pR, (0, 0, 0), thickness + 2, lineType=cv2.LINE_AA)
                cv2.line(img, pT, pB, (0, 0, 0), thickness + 2, lineType=cv2.LINE_AA)
 # lines
            cv2.line(img, pL, pR, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
            cv2.line(img, pT, pB, (255, 255, 255), thickness, lineType=cv2.LINE_AA)

        else:
            raise ValueError("style must be 'filled_circle' or 'cross'")

    return img

# ================================================================
# load（GT）
# ================================================================
with open(GT_JSON, "r", encoding="utf-8") as f:
    gt = json.load(f)

images_by_id   = {im["id"]: im for im in gt["images"]}
images_by_name = {im["file_name"]: im for im in gt["images"]}
ann_by_img     = {}
for ann in gt["annotations"]:
    ann_by_img.setdefault(ann["image_id"], []).append(ann)

# 1images=1GT 
assert all(len(v) == 1 for v in ann_by_img.values()), \
    " 1 images 1 GT 。()。"

# keypointsOKSσ（：0.05）
num_kps = len(gt["categories"][0].get("keypoints", [])) or 10
sigmas  = np.full(num_kps, 0.05, dtype=np.float32)

# ================================================================
# ：inference → 11（OKS） → save
# ================================================================
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

rows = []
results_list = []    # COCO-style results (list of dict; id not required)
pred_anns    = []    # For full COCO: keep the same annotation.id as GT

# ：GTcategory_id（）
cat_id_default = gt["annotations"][0]["category_id"] if len(gt["annotations"]) else 1

for fname, im_meta in images_by_name.items():
    img_path = os.path.join(SRC_DIR, fname)
    if not os.path.exists(img_path):
 # images → 
        continue

    im = cv2.imread(img_path)
    H, W = im.shape[:2]

 # --- GT ---
    gt_ann = ann_by_img[im_meta["id"]][0]
    x, y, w, h = gt_ann["bbox"]  # xywh
    gt_box_xyxy = np.array([x, y, x+w, y+h], dtype=np.float32)
    area = float(max(w, 0) * max(h, 0))

    gk = np.array(gt_ann["keypoints"], dtype=np.float32).reshape(-1, 3)
    gt_kps = gk[:, :2]
 # （v==0） NaN OKS 
    vis = gk[:, 2] > 0
    gt_kps[~vis] = np.nan

 # --- ---
    outs = predictor(im)
    inst = outs["instances"].to("cpu")
    inst = inst[inst.pred_boxes.nonempty()]

    if len(inst) == 0:
 # ：GTdraw、CSVNaN
        canvas = im.copy()
        canvas = _ensure_bgr(canvas)
 # GT: 、bbox
        draw_points_bw(canvas, gt_kps, style="filled_circle", radius=6, thickness=2, halo=True)
        cv2.rectangle(canvas, (int(x), int(y)), (int(x+w), int(y+h)), (255, 255, 255), 2)
        cv2.imwrite(os.path.join(OUT_DIR, f"overlay_{fname}"), canvas)

        rows.append({
            "file": fname, "oks": np.nan, "mean_err_px": np.nan, "norm_err": np.nan
        })
 # COCO：0
        continue

 # --- 1（OKS；area0IoU） ---
    best = None
    best_metric = -1.0
    best_score = None
    best_idx = -1
    for i in range(len(inst)):
        pb = inst.pred_boxes.tensor[i].numpy()     # xyxy
        pk = inst.pred_keypoints.numpy()[i, :, :2] # (K,2)
        score = float(inst.scores[i].item())
        if area > 0:
            m = oks(gt_kps.copy(), pk.copy(), area, sigmas)
        else:
            m = iou_xyxy(gt_box_xyxy, pb)
        if m > best_metric:
            best_metric = m
            best = (pb, pk)
            best_score = score
            best_idx = i

    pred_box_xyxy, pred_kps = best

 # --- compute（） ---
    mask = ~np.isnan(gt_kps[:, 0])
    diffs = np.linalg.norm(pred_kps[mask] - gt_kps[mask], axis=1)
    mean_err_px = float(diffs.mean()) if len(diffs) else np.nan
    diag = np.hypot(w, h) + 1e-12
    norm_err = float(mean_err_px / diag) if not np.isnan(mean_err_px) else np.nan

 # --- drawsave ---
    canvas = im.copy()
    canvas = _ensure_bgr(canvas)
 # GT（ & bbox）
    draw_points_bw(canvas, gt_kps, style="filled_circle", radius=6, thickness=2, halo=True)
    cv2.rectangle(canvas, (int(x), int(y)), (int(x+w), int(y+h)), (255, 255, 255), 2)

 # Pred（ & bbox）
    draw_points_bw(canvas, pred_kps, style="cross", radius=7, thickness=2, halo=True)
    pb = pred_box_xyxy.astype(int)
    cv2.rectangle(canvas, (pb[0], pb[1]), (pb[2], pb[3]), (255, 255, 255), 1)

    cv2.imwrite(os.path.join(OUT_DIR, f"overlay_{fname}"), canvas)

 # --- CSV ---
    row = {
        "file": fname,
        "oks": best_metric if area > 0 else np.nan,
        "mean_err_px": mean_err_px,
        "norm_err": norm_err,
        "score": best_score,
    }
    for i, d in enumerate(diffs):
        row[f"kp{i}_err_px"] = float(d)
    rows.append(row)

 # --- COCO（list；evaluation、id） ---
    if SAVE_COCO_RESULTS:
 # OKS best_idx 
        tmp_inst = inst[[best_idx]]
        coco_items = instances_to_coco_json(tmp_inst, int(im_meta["id"]))
 # ID（）
        for d in coco_items:
            d["category_id"] = cat_id_default
        results_list.extend(coco_items)

 # --- COCO（GT annotation.id） ---
    if SAVE_COCO_FULL:
        gt_ann_id = ann_by_img[im_meta["id"]][0]["id"]  # assumes one GT per image
        # xyxy→xywh
        bx = [float(pb[0]), float(pb[1]), float(pb[2] - pb[0]), float(pb[3] - pb[1])]
 # keypoints（3*K; v2）
        kp_flat = []
        for (px, py) in pred_kps:
            kp_flat.extend([float(px), float(py), 2.0])
        pred_anns.append({
            "id":            gt_ann_id,             # same ID as GT
            "image_id":      im_meta["id"],
            "category_id":   int(cat_id_default),
            "bbox":          bx,
            "area":          float(bx[2] * bx[3]),
            "iscrowd":       0,
            "keypoints":     kp_flat,
            "num_keypoints": int(len(kp_flat) // 3),
            "score":         float(best_score if best_score is not None else 1.0),
        })

# ================================================================
# export（CSV / COCO JSON）
# ================================================================
pd.DataFrame(rows).to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
print(f"✓ overlays → {OUT_DIR}/,  ✓ per-image errors → {CSV_PATH}")

if SAVE_COCO_RESULTS:
    with open(COCO_RESULTS_PATH, "w", encoding="utf-8") as f:
        json.dump(results_list, f)
    print(f"✓ COCOresults: {COCO_RESULTS_PATH} (items={len(results_list)})")

if SAVE_COCO_FULL:
    full_coco = {
        "info":       gt.get("info", {}),
        "licenses":   gt.get("licenses", []),
        "categories": gt["categories"],   # keep keypoint names/skeleton
        "images":     gt["images"],       # preserve file_name/size
        "annotations": pred_anns          # replace with predictions (id identical to GT)
    }
    with open(COCO_FULL_PATH, "w", encoding="utf-8") as f:
        json.dump(full_coco, f)
    print(f"✓ COCO（ID）: {COCO_FULL_PATH} (anns={len(pred_anns)})")

In [ ]:
# Cell purpose: Utility / preparation